In [1]:
import os
os.environ['PYTENSOR_FLAGS'] = 'mode=FAST_COMPILE,optimizer=fast_compile,floatX=float32,cxx='

import numpy as np
import pymc as pm
import pytensor.tensor as pt

In this notebook, we demonstrate the initialization capability of PyMC's ADVI.

We will fit a simple 1d normal distribution using ADVI and compare the initializations.


In [26]:
# run PyMC ADVI algorithm
with pm.Model() as init_model:
    # Define priors
    mu = pm.Normal("mu", 0, 1)
    likelihood = pm.Normal('y', mu=mu, sigma=2, observed=None)
    advi_fit = pm.fit(0, method=pm.ADVI(), start={'mu': 1.0, 'sigma': 1.0})

    mu_idx = [var.name for var in init_model.unobserved_RVs].index("mu")
    m_mu_vb = advi_fit.mean.eval()[mu_idx]
    s_mu_vb = advi_fit.std.eval()[mu_idx]

with pm.Model() as default_model:
    # Define a simple model with a single normal variable
    mu = pm.Normal('mu', mu=0, sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=1, observed = None)

    advi = pm.ADVI(random_seed=20)
    tracker = pm.callbacks.Tracker(
            mean = advi.approx.mean.eval,
            std = advi.approx.std.eval
        )
    default_approx = advi.fit(
            1, callbacks=[tracker],
            obj_optimizer=pm.adagrad(learning_rate=0.01)
        )

Output()

Initialization only


Output()

Finished [100%]: Loss = 0.17482


In [23]:
# we figure out which index mu is - just to make sure we are looking at the correct value
default_mu_idx = [var.name for var in default_model.unobserved_RVs].index("mu")
init_mu_idx = [var.name for var in init_model.unobserved_RVs].index("mu")
default_mu_idx, init_mu_idx

(0, 0)

In [24]:
# This is what the default initialized to
default_approx.mean.eval()[default_mu_idx], default_approx.std.eval()[default_mu_idx]

(np.float32(0.0099999), np.float32(0.6981597))

In [27]:
# This is what the init initialized to, 
# when we told it to use start={'mu': 1.0, 'sigma': 1.0}
advi_fit.mean.eval()[init_mu_idx], advi_fit.std.eval()[init_mu_idx]

(np.float32(0.0), np.float32(0.6931472))